# Reintubation event 생성 및 정리

## 데이터베이스 연결

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [2]:
#pip freeze > requirements.txt

In [3]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('', ),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="",
         ssh_password="", 
         remote_bind_address=('localhost',))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': '',
             'password': '',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()



tunnel connected
database connected


## Reintubation 값 정리

In [4]:
# extubationto reintubationtime 을 계산해야 하는데,
# intcount 가 1인 경우에는 계산할 필요가 없음 reintubationtime = 0 으로 처리
# intcount 가 2 이상인 경우부터 계산
# reintubationtime은 다음행의 intubationtime - 현재 행의 extubationtime  현재행에 결과 도출
# 마지막 행은 reintubationtime 이 없는 것이 정상. 그래서 마지막 행은 reintubationtime = 0 으로 처리


In [6]:
reintubation1 = pd.read_csv('../outputs/reintubation1.csv', parse_dates= ['admittime','intubationtime','extubationtime','dischtime','deathtime'])
df1 = reintubation1.copy()

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8483 entries, 0 to 8482
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   subject_id                  8483 non-null   float64       
 1   hadm_id                     8483 non-null   float64       
 2   admittime                   8483 non-null   datetime64[ns]
 3   intseq                      8483 non-null   int64         
 4   intcount                    8483 non-null   int64         
 5   intubationtime              8483 non-null   datetime64[ns]
 6   extubationtime              8483 non-null   datetime64[ns]
 7   extubationcause             8483 non-null   object        
 8   intubationtoextubationtime  8483 non-null   float64       
 9   int_itemid                  8390 non-null   float64       
 10  int_weight                  8390 non-null   float64       
 11  ext_itemid                  6867 non-null   float64     

In [ ]:
df1[df1['reintubationtime'] < 0]

,subject_id,hadm_id,admittime,intseq,intcount,intubationtime,extubationtime,extubationcause,intubationtoextubationtime,int_itemid,int_weight,ext_itemid,ext_weight,dischtime,deathtime,reintubationtime
18,13605251.0,20025172.0,2139-11-15 17:24:00,1,2,2139-12-31 14:02:00,2140-01-04 11:09:00,Planned Extubation,93.1,224385.0,48.2,227194.0,48.2,2140-01-20 15:15:00,2140-01-20 15:15:00,-93.116667
36,13054784.0,20045781.0,2177-12-16 16:02:00,1,2,2178-03-18 09:50:00,2178-03-24 10:04:00,Planned Extubation,144.2,224385.0,87.2,227194.0,103.5,2178-03-24 13:30:00,2178-03-24 13:30:00,-4.983333
65,10935357.0,20085117.0,2155-08-23 17:05:00,1,2,2155-08-24 08:45:00,2155-09-14 00:31:00,Planned Extubation,495.8,224385.0,74.5,227194.0,74.5,2155-10-04 15:40:00,NaT,-128.266667
67,16691089.0,20089438.0,2179-12-23 16:15:00,1,2,2179-12-23 18:09:00,2179-12-24 13:15:00,Planned Extubation,19.1,224385.0,60.9,227194.0,60.9,2180-01-19 17:37:00,NaT,-19.100000
132,10562205.0,20149612.0,2149-04-29 23:19:00,1,2,2149-04-30 09:30:00,2149-05-07 09:10:00,Planned Extubation,167.7,224385.0,98.0,227194.0,98.0,2149-05-11 15:20:00,NaT,-28.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8288,11111901.0,29789360.0,2163-12-23 12:54:00,2,3,2164-01-02 03:00:00,2164-01-10 09:56:00,Planned Extubation,198.9,224385.0,73.8,227194.0,73.8,2164-02-12 17:43:00,NaT,-143.933333
8290,11495677.0,29790204.0,2168-02-26 13:40:00,1,2,2168-02-26 15:00:00,2168-03-07 10:26:00,Planned Extubation,235.4,224385.0,90.0,227194.0,90.0,2168-03-22 18:56:00,NaT,-14.183333
8319,10332371.0,29819591.0,2149-02-19 19:02:00,1,2,2149-03-06 11:00:00,2149-03-21 12:04:00,Planned Extubation,361.1,224385.0,66.0,227194.0,66.0,2149-03-28 16:00:00,NaT,-197.066667
8470,19652570.0,29987115.0,2148-02-16 17:42:00,1,2,2148-02-17 14:00:00,2148-02-20 18:57:00,Planned Extubation,77.0,224385.0,67.0,227194.0,67.0,2148-02-20 19:00:00,2148-02-20 19:00:00,-6.600000


In [ ]:
# #intubationtoextubationtime 이 NaT 인 에러들이 속한 hadm_id들의 데이터를 확인
error_hadm_ids_df1 = df1[df1['reintubationtime'] < 0]['hadm_id'].unique()
error_rows_df1 = df1[df1['hadm_id'].isin(error_hadm_ids_df1)]
display(error_rows_df1)

,subject_id,hadm_id,admittime,intseq,intcount,intubationtime,extubationtime,extubationcause,intubationtoextubationtime,int_itemid,int_weight,ext_itemid,ext_weight,dischtime,deathtime,reintubationtime
18,13605251.0,20025172.0,2139-11-15 17:24:00,1,2,2139-12-31 14:02:00,2140-01-04 11:09:00,Planned Extubation,93.1,224385.0,48.2,227194.0,48.2,2140-01-20 15:15:00,2140-01-20 15:15:00,-93.116667
19,13605251.0,20025172.0,2139-11-15 17:24:00,2,2,2139-12-31 14:02:00,2140-01-21 16:45:00,Planned Extubation,506.7,224385.0,48.2,227194.0,48.2,2140-01-20 15:15:00,2140-01-20 15:15:00,0.000000
36,13054784.0,20045781.0,2177-12-16 16:02:00,1,2,2178-03-18 09:50:00,2178-03-24 10:04:00,Planned Extubation,144.2,224385.0,87.2,227194.0,103.5,2178-03-24 13:30:00,2178-03-24 13:30:00,-4.983333
37,13054784.0,20045781.0,2177-12-16 16:02:00,2,2,2178-03-24 05:05:00,2178-03-24 10:04:00,Planned Extubation,5.0,224385.0,103.5,227194.0,103.5,2178-03-24 13:30:00,2178-03-24 13:30:00,0.000000
65,10935357.0,20085117.0,2155-08-23 17:05:00,1,2,2155-08-24 08:45:00,2155-09-14 00:31:00,Planned Extubation,495.8,224385.0,74.5,227194.0,74.5,2155-10-04 15:40:00,NaT,-128.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8320,10332371.0,29819591.0,2149-02-19 19:02:00,2,2,2149-03-13 07:00:00,2149-03-21 12:04:00,Planned Extubation,197.1,224385.0,66.0,227194.0,66.0,2149-03-28 16:00:00,NaT,0.000000
8470,19652570.0,29987115.0,2148-02-16 17:42:00,1,2,2148-02-17 14:00:00,2148-02-20 18:57:00,Planned Extubation,77.0,224385.0,67.0,227194.0,67.0,2148-02-20 19:00:00,2148-02-20 19:00:00,-6.600000
8471,19652570.0,29987115.0,2148-02-16 17:42:00,2,2,2148-02-20 12:21:00,2148-02-20 18:57:00,Planned Extubation,6.6,224385.0,67.0,227194.0,67.0,2148-02-20 19:00:00,2148-02-20 19:00:00,0.000000
8478,12136570.0,29992577.0,2147-06-13 21:51:00,1,2,2147-06-14 00:36:00,2147-06-24 13:30:00,Planned Extubation,252.9,224385.0,117.0,227194.0,117.0,2147-07-12 18:09:00,NaT,-30.166667


In [ ]:
error_rows_df1[0:30]

,subject_id,hadm_id,admittime,intseq,intcount,intubationtime,extubationtime,extubationcause,intubationtoextubationtime,int_itemid,int_weight,ext_itemid,ext_weight,dischtime,deathtime,reintubationtime
18,13605251.0,20025172.0,2139-11-15 17:24:00,1,2,2139-12-31 14:02:00,2140-01-04 11:09:00,Planned Extubation,93.1,224385.0,48.2,227194.0,48.2,2140-01-20 15:15:00,2140-01-20 15:15:00,-93.116667
19,13605251.0,20025172.0,2139-11-15 17:24:00,2,2,2139-12-31 14:02:00,2140-01-21 16:45:00,Planned Extubation,506.7,224385.0,48.2,227194.0,48.2,2140-01-20 15:15:00,2140-01-20 15:15:00,0.000000
36,13054784.0,20045781.0,2177-12-16 16:02:00,1,2,2178-03-18 09:50:00,2178-03-24 10:04:00,Planned Extubation,144.2,224385.0,87.2,227194.0,103.5,2178-03-24 13:30:00,2178-03-24 13:30:00,-4.983333
37,13054784.0,20045781.0,2177-12-16 16:02:00,2,2,2178-03-24 05:05:00,2178-03-24 10:04:00,Planned Extubation,5.0,224385.0,103.5,227194.0,103.5,2178-03-24 13:30:00,2178-03-24 13:30:00,0.000000
65,10935357.0,20085117.0,2155-08-23 17:05:00,1,2,2155-08-24 08:45:00,2155-09-14 00:31:00,Planned Extubation,495.8,224385.0,74.5,227194.0,74.5,2155-10-04 15:40:00,NaT,-128.266667
66,10935357.0,20085117.0,2155-08-23 17:05:00,2,2,2155-09-08 16:15:00,2155-09-14 00:31:00,Planned Extubation,128.3,224385.0,74.5,227194.0,74.5,2155-10-04 15:40:00,NaT,0.000000
67,16691089.0,20089438.0,2179-12-23 16:15:00,1,2,2179-12-23 18:09:00,2179-12-24 13:15:00,Planned Extubation,19.1,224385.0,60.9,227194.0,60.9,2180-01-19 17:37:00,NaT,-19.100000
68,16691089.0,20089438.0,2179-12-23 16:15:00,2,2,2179-12-23 18:09:00,2179-12-31 18:20:00,Planned Extubation,192.2,224385.0,60.9,227194.0,60.9,2180-01-19 17:37:00,NaT,0.000000
132,10562205.0,20149612.0,2149-04-29 23:19:00,1,2,2149-04-30 09:30:00,2149-05-07 09:10:00,Planned Extubation,167.7,224385.0,98.0,227194.0,98.0,2149-05-11 15:20:00,NaT,-28.600000
133,10562205.0,20149612.0,2149-04-29 23:19:00,2,2,2149-05-06 04:34:00,2149-05-07 09:10:00,Planned Extubation,28.6,224385.0,98.0,227194.0,98.0,2149-05-11 15:20:00,NaT,0.000000


In [ ]:
def group_time1(group):
    group = group.sort_index()
    
    for idx in group.index:
        # 현재 row와 다음 row를 비교하기 위해 다음 row의 정보를 가져온다.
        # 최대/최소 인덱스가 존재하지 않을 경우를 처리합니다.
        prev_idx = group.index[group.index < idx].max() if not group.index[group.index < idx].empty else None
        next_idx = group.index[group.index > idx].min() if not group.index[group.index > idx].empty else None

        prev_row = group.loc[prev_idx] if prev_idx is not None else None
        next_row = group.loc[next_idx] if next_idx is not None else None
        
        # 첫 번째 행을 처리합니다.
        if prev_row is None and pd.isnull(group.loc[idx, 'extubationtime']):
            if next_row is not None:
                if group.loc[idx, 'reintubationtime'] < 0:
                    # intubationtime 이 중복될 때
                    if group.loc[idx, 'intubationtime'] == next_row['intubationtime'] and group.loc[idx, 'extubationtime'] != next_row['extubationtime']:
                        group.loc[idx, 'intubationtime'] = group.loc[idx,'admittime']
                        group.loc[idx, 'extubationcause'] = 'intubation replaced'

                    # extubationtime 이 중복될 때
                    if next_row['intubationtime'] < group.loc[idx, 'extubationtime'] and group.loc[idx, 'extubationtime'] == next_row['extubationtime']:
                        group.loc[idx, 'extubationtime'] = next_row['intubationtime']
                        group.loc[idx, 'extubationcause'] = 'extubation replace'

    return group

In [ ]:
df2 = df1.groupby('hadm_id').apply(group_time1).reset_index(drop=True)
df2

,subject_id,hadm_id,admittime,intseq,intcount,intubationtime,extubationtime,extubationcause,intubationtoextubationtime,int_itemid,int_weight,ext_itemid,ext_weight,dischtime,deathtime,reintubationtime
0,14577567.0,20001361.0,2143-05-04 14:55:00,1,1,2143-05-07 21:07:00,2143-05-08 16:21:00,Planned Extubation,19.2,224385.0,102.0,227194.0,102.0,2143-05-18 16:58:00,NaT,0.000000
1,19657904.0,20004357.0,2157-08-05 10:57:00,1,1,2157-08-05 19:00:00,2157-08-07 13:28:00,Planned Extubation,42.5,224385.0,96.2,227194.0,96.2,2157-08-12 18:00:00,NaT,0.000000
2,19244907.0,20005119.0,2112-11-08 20:25:00,1,1,2112-11-12 09:10:00,2112-12-16 17:42:00,dischtime,824.5,224385.0,57.2,NaN,NaN,2112-12-16 17:42:00,NaT,0.000000
3,17038060.0,20005241.0,2178-01-22 15:51:00,1,1,2178-01-26 11:54:00,2178-01-26 18:22:00,Planned Extubation,6.5,224385.0,65.0,227194.0,65.0,2178-01-31 15:13:00,NaT,0.000000
4,17844820.0,20014219.0,2142-10-06 13:37:00,1,1,2142-10-09 22:45:00,2142-10-20 10:48:00,Planned Extubation,252.0,224385.0,96.6,227194.0,96.6,2142-11-11 18:53:00,NaT,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8478,12136570.0,29992577.0,2147-06-13 21:51:00,1,2,2147-06-14 00:36:00,2147-06-24 13:30:00,Planned Extubation,252.9,224385.0,117.0,227194.0,117.0,2147-07-12 18:09:00,NaT,-30.166667
8479,12136570.0,29992577.0,2147-06-13 21:51:00,2,2,2147-06-23 07:20:00,2147-06-24 13:30:00,Planned Extubation,30.2,224385.0,117.0,227194.0,117.0,2147-07-12 18:09:00,NaT,0.000000
8480,13997063.0,29995505.0,2123-03-05 19:29:00,1,1,2123-03-06 23:42:00,2123-03-29 15:00:00,dischtime,543.3,224385.0,122.3,NaN,NaN,2123-03-29 15:00:00,NaT,0.000000
8481,17660536.0,29997844.0,2125-02-16 20:10:00,1,1,2125-02-26 16:25:00,2125-03-03 16:21:00,dischtime,119.9,224385.0,98.0,NaN,NaN,2125-03-03 16:21:00,NaT,0.000000


In [ ]:
def calculate_reintubation_time(group):
    # 그룹을 인덱스 순서대로 정렬 (중요함, 인덱스가 비순차적일 수 있으므로)
    group = group.sort_index()
    
    # `reintubationtime` 컬럼을 0으로 초기화
    group['reintubationtime'] = 0
    
    # 마지막 행을 제외한 모든 행을 반복
    for i in range(len(group) - 1):
        current_row = group.iloc[i]
        next_row = group.iloc[i + 1]
        
        # `intcount`가 2 이상인 경우에만 계산
        if current_row['intcount'] >= 2:
            # 다음 행의 `intubationtime`에서 현재 행의 `extubationtime`을 뺀다
            group.at[current_row.name, 'reintubationtime'] = (next_row['intubationtime'] - current_row['extubationtime']).total_seconds() / 3600  # 초 단위로 계산한 다음 시간으로 변환
    
    # 마지막 행 처리
    # 마지막 행에 대해서는 `reintubationtime`을 0으로 설정 (이미 초기화 되었으므로 생략 가능)
    
    return group

# 이 함수를 각 그룹(예: `subject_id`별로 그룹화된 데이터프레임)에 적용합니다.
# 예: df.groupby('subject_id').apply(calculate_reintubation_time)

In [ ]:
# 데이터프레임이 'df'라고 가정하고, 'subject_id'로 그룹화한 다음 함수를 적용
df2 = df2.groupby('hadm_id').apply(calculate_reintubation_time).reset_index(drop=True)

In [ ]:
# #intubationtoextubationtime 이 NaT 인 에러들이 속한 hadm_id들의 데이터를 확인
error_hadm_ids_df2 = df2[df2['reintubationtime'] < 0]['hadm_id'].unique()
error_rows_df2 = df2[df2['hadm_id'].isin(error_hadm_ids_df2)]
display(error_rows_df2)

,subject_id,hadm_id,admittime,intseq,intcount,intubationtime,extubationtime,extubationcause,intubationtoextubationtime,int_itemid,int_weight,ext_itemid,ext_weight,dischtime,deathtime,reintubationtime
18,13605251.0,20025172.0,2139-11-15 17:24:00,1,2,2139-12-31 14:02:00,2140-01-04 11:09:00,Planned Extubation,93.1,224385.0,48.2,227194.0,48.2,2140-01-20 15:15:00,2140-01-20 15:15:00,-93.116667
19,13605251.0,20025172.0,2139-11-15 17:24:00,2,2,2139-12-31 14:02:00,2140-01-21 16:45:00,Planned Extubation,506.7,224385.0,48.2,227194.0,48.2,2140-01-20 15:15:00,2140-01-20 15:15:00,0.000000
36,13054784.0,20045781.0,2177-12-16 16:02:00,1,2,2178-03-18 09:50:00,2178-03-24 10:04:00,Planned Extubation,144.2,224385.0,87.2,227194.0,103.5,2178-03-24 13:30:00,2178-03-24 13:30:00,-4.983333
37,13054784.0,20045781.0,2177-12-16 16:02:00,2,2,2178-03-24 05:05:00,2178-03-24 10:04:00,Planned Extubation,5.0,224385.0,103.5,227194.0,103.5,2178-03-24 13:30:00,2178-03-24 13:30:00,0.000000
65,10935357.0,20085117.0,2155-08-23 17:05:00,1,2,2155-08-24 08:45:00,2155-09-14 00:31:00,Planned Extubation,495.8,224385.0,74.5,227194.0,74.5,2155-10-04 15:40:00,NaT,-128.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8320,10332371.0,29819591.0,2149-02-19 19:02:00,2,2,2149-03-13 07:00:00,2149-03-21 12:04:00,Planned Extubation,197.1,224385.0,66.0,227194.0,66.0,2149-03-28 16:00:00,NaT,0.000000
8470,19652570.0,29987115.0,2148-02-16 17:42:00,1,2,2148-02-17 14:00:00,2148-02-20 18:57:00,Planned Extubation,77.0,224385.0,67.0,227194.0,67.0,2148-02-20 19:00:00,2148-02-20 19:00:00,-6.600000
8471,19652570.0,29987115.0,2148-02-16 17:42:00,2,2,2148-02-20 12:21:00,2148-02-20 18:57:00,Planned Extubation,6.6,224385.0,67.0,227194.0,67.0,2148-02-20 19:00:00,2148-02-20 19:00:00,0.000000
8478,12136570.0,29992577.0,2147-06-13 21:51:00,1,2,2147-06-14 00:36:00,2147-06-24 13:30:00,Planned Extubation,252.9,224385.0,117.0,227194.0,117.0,2147-07-12 18:09:00,NaT,-30.166667


In [ ]:
#tunnel.close()